In [272]:
import cirq
import numpy as np
import random
import sympy
from math import pi
%matplotlib inline

np.random.seed(42)

In [441]:
def U_phi(q, W):
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=W[i])
        yield rot(q[i])
    for i in range(len(q)):
        if i < len(q)-1:
            rot = cirq.ZPowGate(exponent=(pi-W[i])*(pi-W[i+1]))
            yield rot.on(q[i+1]).controlled_by(q[i])
        else:
            rot = cirq.ZPowGate(exponent=(pi-W[i])*(pi-W[0]))
            yield rot.on(q[0]).controlled_by(q[i])
            
def fancy_U(q, W):
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)
    for i in range(len(q)):
        yield cirq.H(q[i])
    yield U_phi(q, W)

def W_theta(q, theta):
    for i in range(len(q)):
        if i < len(q)-1:
            yield cirq.SWAP.on(q[i+1],q[i])
        else:
            yield cirq.SWAP.on(q[0],q[i])
    for i in range(len(q)):
        rot = cirq.ZPowGate(exponent=theta[i])
        yield rot(q[i])

def measure(q):
    for i in range(len(q)):
        yield cirq.measure(q[i], key=str(i))

In [379]:
def circuit(q, W, theta):
    yield fancy_U(q, W)
    for i in range(len(theta)):
        if i == 0:
            for j in range(len(q)):
                rot = cirq.ZPowGate(exponent=theta[i][j])
                yield rot(q[j])
        else:
            yield W_theta(q, theta[i])
    yield measure(q)

In [449]:
nr_qubits = 4
nr_layers = 1
batch_size = 5
repititions = 100

#Set up input and output qubits.
qubits = [cirq.GridQubit(i, 0) for i in range(nr_qubits)]

data = np.loadtxt("./parity.txt")
X = data[:, :-1]
Y = data[:, -1]

init_rot = [np.random.randn(nr_qubits) for i in range(nr_layers+1)]

simulator = cirq.Simulator()
acc = 0

for i in range(len(X)):
    c = cirq.Circuit()
    c.append(circuit(qubits, X[i], init_rot))
    results = simulator.run(c)#, repetitions)
    count = 0
    for j in range(nr_qubits):
        count = count + results.histogram(key=str(j))[0]
    print("true:",Y[i],"predict:",count%2,"equal:",(Y[i]==count%2))
    acc = acc + int(Y[i]==count%2)
print("accuracy:",acc/len(X))
# print('Circuit:')
# print(c)

true: 0.0 predict: 0 equal: True
true: 1.0 predict: 0 equal: False
true: 1.0 predict: 0 equal: False
true: 0.0 predict: 1 equal: False
true: 1.0 predict: 1 equal: True
true: 0.0 predict: 0 equal: True
true: 0.0 predict: 1 equal: False
true: 1.0 predict: 0 equal: False
true: 1.0 predict: 0 equal: False
true: 0.0 predict: 1 equal: False
true: 0.0 predict: 1 equal: False
true: 1.0 predict: 0 equal: False
true: 0.0 predict: 0 equal: True
true: 1.0 predict: 1 equal: True
true: 1.0 predict: 1 equal: True
true: 0.0 predict: 0 equal: True
accuracy: 0.4375


In [429]:
# def rot_x_layer(length, half_turns):
#     """Yields X rotations by half_turns on a square grid of given length."""
#     rot1 = cirq.ZPowGate(exponent=half_turns)
#     rot2 = cirq.XPowGate(exponent=half_turns)
#     rot3 = cirq.ZPowGate(exponent=half_turns)
#     for i in range(length):
#         for j in range(length):
#             yield rot(cirq.GridQubit(i, j))

# circuit = cirq.Circuit()
# circuit.append(rot_x_layer(2, 0.1))
# print(circuit)

In [432]:
# def rand2d(rows, cols):
#     return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

# def random_instance(length):
#     # transverse field terms
#     h = rand2d(length, length)
#     # links within a row
#     jr = rand2d(length - 1, length)
#     # links within a column
#     jc = rand2d(length, length - 1)
#     return (h, jr, jc)

# h, jr, jc = random_instance(3)
# print('transverse fields: {}'.format(h))
# print('row j fields: {}'.format(jr))
# print('column j fields: {}'.format(jc))

In [433]:
# def rot_z_layer(h, half_turns):
#     """Yields Z rotations by half_turns conditioned on the field h."""
#     gate = cirq.ZPowGate(exponent=half_turns)
#     for i, h_row in enumerate(h):
#         for j, h_ij in enumerate(h_row):
#             if h_ij == 1:
#                 yield gate(cirq.GridQubit(i, j))

# def rot_11_layer(jr, jc, half_turns):
#     """Yields rotations about |11> conditioned on the jr and jc fields."""
#     gate = cirq.CZPowGate(exponent=half_turns)    
#     for i, jr_row in enumerate(jr):
#         for j, jr_ij in enumerate(jr_row):
#             if jr_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i + 1, j))
#             yield gate(cirq.GridQubit(i, j),
#                        cirq.GridQubit(i + 1, j))
#             if jr_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i + 1, j))

#     for i, jc_row in enumerate(jc):
#         for j, jc_ij in enumerate(jc_row):
#             if jc_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i, j + 1))
#             yield gate(cirq.GridQubit(i, j),
#                        cirq.GridQubit(i, j + 1))
#             if jc_ij == -1:
#                 yield cirq.X(cirq.GridQubit(i, j))
#                 yield cirq.X(cirq.GridQubit(i, j + 1))
                
# def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
#     length = len(h)
#     yield rot_x_layer(length, x_half_turns)
#     yield rot_z_layer(h, h_half_turns)
#     yield rot_11_layer(jr, jc, j_half_turns)

In [434]:
# h, jr, jc = random_instance(3)

# circuit = cirq.Circuit()    
# circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
#                strategy=cirq.InsertStrategy.EARLIEST)
# print(circuit)

In [435]:
# simulator = cirq.Simulator()
# circuit = cirq.Circuit()    
# circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
# circuit.append(cirq.measure(*qubits, key='x'))
# results = simulator.run(circuit, repetitions=100)
# print(results.histogram(key='x'))

In [436]:
# def energy_func(length, h, jr, jc):
#     def energy(measurements):
#         # Reshape measurement into array that matches grid shape.
#         meas_list_of_lists = [measurements[i * length:(i + 1) * length]
#                               for i in range(length)]
#         # Convert true/false to +1/-1.
#         pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

#         tot_energy = np.sum(pm_meas * h)
#         for i, jr_row in enumerate(jr):
#             for j, jr_ij in enumerate(jr_row):
#                 tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
#         for i, jc_row in enumerate(jc):
#             for j, jc_ij in enumerate(jc_row):
#                 tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
#         return tot_energy
#     return energy
# print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

In [437]:
# def obj_func(result):
#     energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
#     return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
# print('Value of the objective function {}'.format(obj_func(results)))

In [438]:
# circuit = cirq.Circuit()
# alpha = sympy.Symbol('alpha')
# beta = sympy.Symbol('beta')
# gamma = sympy.Symbol('gamma')
# circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
# circuit.append(cirq.measure(*qubits, key='x'))
# print(circuit)

In [439]:
# resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
# resolved_circuit = cirq.resolve_parameters(circuit, resolver)

# sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
#          * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
#          * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
# results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
# for result in results:
#     print(result.params.param_dict, obj_func(result))

In [440]:
# sweep_size = 10
# sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=10)
#          * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=10)
#          * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=10))
# results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

# min = None
# min_params = None
# for result in results:
#     value = obj_func(result)
#     if min is None or value < min:
#         min = value
#         min_params = result.params
# print('Minimum objective value is {}.'.format(min))